# Vi lager et program som lar brukeren velge et sted i Norge og se den historiske værdataene til dette stedet. 
# Programmet henter informasjon fra den nærmeste værstasjonen til det valgte stedet og visualiserer dataene i grafer
# Programmet viser også en beregning for hvordan framtidig vær kommer til å se ut.
# All data er hentet fra MET som er en offentlig kilde sponset fra den norske stat
e9a6e9b1b9be44fab4720c2c96ae9c48

In [40]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from opencage.geocoder import OpenCageGeocode
from geopy.distance import geodesic

# Sett opp OpenCageGeocode med API-nøkkelen din
api_key = 'e9a6e9b1b9be44fab4720c2c96ae9c48'  # Sett inn din OpenCage API-nøkkel
geolocator = OpenCageGeocode(api_key)

# Henter inn lokasjon fra brukeren
location_input = input("Enter the location (e.g., Bergen, Oslo, Trondheim, Stavanger, Tromsø): ")

# Geokoder lokasjonen for å få koordinatene
location = geolocator.geocode(location_input)

# Sjekker om geokodingen var vellykket
if location:
    # Henter ut koordinatene
    location_coords = (location[0]['geometry']['lat'], location[0]['geometry']['lng'])
else:
    raise ValueError("Could not find coordinates for the location. Please check the input.")

# Vår klient-ID for å hente data fra Frost API
client_id = '2e243d34-57bc-42b4-8095-239991af5353'

# Definerer endepunkt og parametere for stasjoner
stations_endpoint = 'https://frost.met.no/sources/v0.jsonld'
stations_parameters = {
    'types': 'SensorSystem'
}

# Henter ut HTTP-responsen og lagrer den som JSON
r = requests.get(stations_endpoint, stations_parameters, auth=(client_id, ''))
stations_json = r.json()

# Henter ut stasjonsinformasjonen
stations = stations_json['data']

# Lager et dictionary med stasjons-ID som nøkkel og koordinater som verdi
weather_stations_coords = {station['id']: (station['geometry']['coordinates'][1], station['geometry']['coordinates'][0]) for station in stations if 'geometry' in station}

# Regner ut avstanden til værstasjonene fra lokasjonen
distances = {station: geodesic(location_coords, coords).km for station, coords in weather_stations_coords.items()}

# Sorterer værstasjonene etter avstand til lokasjonen
sorted_stations = sorted(distances, key=distances.get)

# Henter ut dagens dato og datoen 90 dager tilbake
now = datetime.now()
offset = now - timedelta(days=90)
current_date = now.strftime("%Y-%m-%d")
historic_date = offset.strftime("%Y-%m-%d")

# Definerer endepunkt og parametere for observasjoner
observations_endpoint = 'https://frost.met.no/observations/v0.jsonld'

def fetch_data_by_element(stations, elements, max_attempts = 15):
    """
    Fetch data for each element (e.g., temperature, precipitation, wind speed) from the nearest weather stations.
    Tries up to max_attempts stations for each element and returns the data for the first successful attempt.
    """
    all_data = {}
    
    for element in elements:
        print(f"Henter data for element: {element}")
        for attempt in range(min(max_attempts, len(stations))):
            parameters = {
                'sources': stations[attempt],
                'elements': element,
                'referencetime': f'{historic_date}/{current_date}',
            }
            r = requests.get(observations_endpoint, parameters, auth=(client_id, ''))
            json = r.json()

            if r.status_code == 200 and 'data' in json:
                data = json['data']
                if data:
                    print(f"Data retrieved for {element} from station {stations[attempt]}!")
                    all_data[element] = data
                    break
            else:
                print(f"Error retrieving {element} data from station {stations[attempt]}. Trying next station...")
        
        # Hvis ingen data blir funnet for elementet, logges en melding
        if element not in all_data:
            print(f"No data found for {element} from any of the attempted stations.")
    
    return all_data

# Elementene vi ønsker å hente data for
elements = [
    'mean(air_temperature P1D)', 
    'sum(precipitation_amount P1D)', 
    'mean(wind_speed P1D)'
]

# Henter data for lokasjonen
data = fetch_data_by_element(sorted_stations, elements)

# Samler de viktige dataene i en DataFrame
df_list = []

for element, data_element in data.items():
    for item in data_element:
        row = pd.DataFrame(item['observations'])
        row['referenceTime'] = item['referenceTime']
        row['sourceId'] = item['sourceId']
        row['elementId'] = element
        df_list.append(row)

# Effektivt å koble sammen alle DataFrames samtidig
df = pd.concat(df_list, ignore_index=True)

# Lagrer dataene i en CSV-fil (overskriver eksisterende fil)
csv_filename = "../src/weather_data.csv"
df.to_csv(csv_filename, index=False)
print(f"Data saved to {csv_filename}")


Henter data for element: mean(air_temperature P1D)
Error retrieving mean(air_temperature P1D) data from station SN18317. Trying next station...
Error retrieving mean(air_temperature P1D) data from station SN18645. Trying next station...
Data retrieved for mean(air_temperature P1D) from station SN18690!
Henter data for element: sum(precipitation_amount P1D)
Data retrieved for sum(precipitation_amount P1D) from station SN18317!
Henter data for element: mean(wind_speed P1D)
Error retrieving mean(wind_speed P1D) data from station SN18317. Trying next station...
Error retrieving mean(wind_speed P1D) data from station SN18645. Trying next station...
Error retrieving mean(wind_speed P1D) data from station SN18690. Trying next station...
Error retrieving mean(wind_speed P1D) data from station SN18310. Trying next station...
Error retrieving mean(wind_speed P1D) data from station SN499999010. Trying next station...
Error retrieving mean(wind_speed P1D) data from station SN18162. Trying next sta

In [42]:
import pandas as pd
import plotly.graph_objects as go

# Bruker den eksisterende variabelen location_input som bynavn
city_name = location_input

# Konverterer referansetiden til datetime
df['referenceTime'] = pd.to_datetime(df['referenceTime'])

# Separerer dataene etter elementId
temperature_data = df[df['elementId'] == 'mean(air_temperature P1D)']
precipitation_data = df[df['elementId'] == 'sum(precipitation_amount P1D)']
wind_speed_data = df[df['elementId'] == 'mean(wind_speed P1D)']

# Definerer farger for grafene
color_location = '#1f77b4'  # Blå for lokasjonen

# Beregninger for temperatur
median_temp = temperature_data['value'].median()
std_temp = temperature_data['value'].std()

# Beregninger for nedbør
median_precip = precipitation_data['value'].median()
std_precip = precipitation_data['value'].std()

# Beregninger for vindhastighet
median_wind = wind_speed_data['value'].median()
std_wind = wind_speed_data['value'].std()

# Temperaturfigur
fig_temp = go.Figure()

# Temperaturdata
fig_temp.add_trace(go.Scatter(
    x=temperature_data['referenceTime'],
    y=temperature_data['value'],
    mode='lines',
    name=f'Temperatur i {city_name}',
    line=dict(color=color_location)
))

# Gjennomsnittstemperatur
mean_temp = temperature_data['value'].mean()
fig_temp.add_trace(go.Scatter(
    x=temperature_data['referenceTime'],
    y=[mean_temp] * len(temperature_data['referenceTime']),
    mode='lines',
    line=dict(dash='dash', color=color_location),
    name=f'Gjennomsnittstemperatur i {city_name}'
))

# Tilpasser layout for temperatur
fig_temp.update_layout(
    title=f'Temperatur over tid i {city_name}',
    xaxis_title='Dato',
    yaxis_title='Temperatur (°C)',
    legend_title='Temperatur'
)

# Viser temperaturfigur
fig_temp.show()

# Utskrift av resultatene for temperatur
print("Temperatur:")
print(f"Median: {median_temp:.2f}, Standardavvik: {std_temp:.2f}")

# Nedbørsfigur
fig_precip = go.Figure()

# Nedbørsdata
fig_precip.add_trace(go.Scatter(
    x=precipitation_data['referenceTime'],
    y=precipitation_data['value'],
    mode='lines',
    name=f'Nedbør i {city_name}',
    line=dict(color=color_location)
))

# Gjennomsnittsnedbør
mean_precip = precipitation_data['value'].mean()
fig_precip.add_trace(go.Scatter(
    x=precipitation_data['referenceTime'],
    y=[mean_precip] * len(precipitation_data['referenceTime']),
    mode='lines',
    line=dict(dash='dash', color=color_location),
    name=f'Gjennomsnittsnedbør i {city_name}'
))

# Tilpasser layout for nedbør
fig_precip.update_layout(
    title=f'Nedbør over tid i {city_name}',
    xaxis_title='Dato',
    yaxis_title='Nedbør (mm)',
    legend_title='Nedbør'
)

# Viser nedbørsfigur
fig_precip.show()

# Utskrift av resultatene for nedbør
print("\nNedbør:")
print(f"Median: {median_precip:.2f}, Standardavvik: {std_precip:.2f}")

# Vindhastighetsfigur
fig_wind = go.Figure()

# Vindhastighetsdata
fig_wind.add_trace(go.Scatter(
    x=wind_speed_data['referenceTime'],
    y=wind_speed_data['value'],
    mode='lines',
    name=f'Vindhastighet i {city_name}',
    line=dict(color=color_location)
))

# Gjennomsnittsvindhastighet
mean_wind = wind_speed_data['value'].mean()
fig_wind.add_trace(go.Scatter(
    x=wind_speed_data['referenceTime'],
    y=[mean_wind] * len(wind_speed_data['referenceTime']),
    mode='lines',
    line=dict(dash='dash', color=color_location),
    name=f'Gjennomsnittsvindhastighet i {city_name}'
))

# Tilpasser layout for vindhastighet
fig_wind.update_layout(
    title=f'Vindhastighet over tid i {city_name}',
    xaxis_title='Dato',
    yaxis_title='Vindhastighet (m/s)',
    legend_title='Vindhastighet'
)

# Viser vindhastighetsfigur
fig_wind.show()

# Utskrift av resultatene for vindhastighet
print("\nVindhastighet:")
print(f"Median: {median_wind:.2f}, Standardavvik: {std_wind:.2f}")

Temperatur:
Median: 1.15, Standardavvik: 3.21



Nedbør:
Median: 0.00, Standardavvik: 2.50



Vindhastighet:
Median: 2.10, Standardavvik: 1.11


In [ ]:
#plotter figurene i nettleseren
fig_temp.show(renderer="browser")
fig_precip.show(renderer="browser")
fig_wind.show(renderer="browser")